In [71]:
from ucimlrepo import fetch_ucirepo 
import pandas as pd
from scipy.io import arff

Data using import from website api

In [72]:
#from ucimlrepo import fetch_ucirepo 
#import pandas as pd
#from scipy.io import arff
#
#polish_companies_bankruptcy = fetch_ucirepo(id=365) 
#X: pd.DataFrame
#y: pd.DataFrame
#
#
##data (as pandas dataframes) 
#X = polish_companies_bankruptcy.data.features 
#y = polish_companies_bankruptcy.data.targets
#
#print(polish_companies_bankruptcy.metadata) 
## variable information 
#print(polish_companies_bankruptcy.variables) 
#
#print("Data:")
#
#print("Features")
#X
#print("Targets")
#y

Data using downloaded files

In [73]:
from ucimlrepo import fetch_ucirepo 
import pandas as pd
from scipy.io import arff
 
X: pd.DataFrame
y: pd.DataFrame

arff_file_year_5 = arff.loadarff('PolishBankruptcy/5year.arff')

df = pd.DataFrame(arff_file_year_5[0])
print("data frame")
df

data frame


,Attr1,Attr2,Attr3,Attr4,Attr5,Attr6,Attr7,Attr8,Attr9,Attr10,...,Attr56,Attr57,Attr58,Attr59,Attr60,Attr61,Attr62,Attr63,Attr64,class
0,0.088238,0.55472,0.011340,1.02050,-66.5200,0.342040,0.109490,0.57752,1.08810,0.320360,...,0.080955,0.275430,0.91905,0.002024,7.2711,4.7343,142.760,2.5568,3.25970,b'0'
1,-0.006202,0.48465,0.232980,1.59980,6.1825,0.000000,-0.006202,1.06340,1.27570,0.515350,...,-0.028591,-0.012035,1.00470,0.152220,6.0911,3.2749,111.140,3.2841,3.37000,b'0'
2,0.130240,0.22142,0.577510,3.60820,120.0400,0.187640,0.162120,3.05900,1.14150,0.677310,...,0.123960,0.192290,0.87604,0.000000,8.7934,2.9870,71.531,5.1027,5.61880,b'0'
3,-0.089951,0.88700,0.269270,1.52220,-55.9920,-0.073957,-0.089951,0.12740,1.27540,0.113000,...,0.418840,-0.796020,0.59074,2.878700,7.6524,3.3302,147.560,2.4735,5.92990,b'0'
4,0.048179,0.55041,0.107650,1.24370,-22.9590,0.000000,0.059280,0.81682,1.51500,0.449590,...,0.240400,0.107160,0.77048,0.139380,10.1180,4.0950,106.430,3.4294,3.36220,b'0'
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5905,0.012898,0.70621,0.038857,1.17220,-18.9070,0.000000,0.013981,0.41600,1.67680,0.293790,...,0.020169,0.043904,1.01220,1.259400,13.4720,12.4320,49.117,7.4313,2.27990,b'1'
5906,-0.578050,0.96702,-0.800850,0.16576,-67.3650,-0.578050,-0.578050,-0.40334,0.93979,-0.390040,...,-0.064073,1.482000,1.06410,-0.018084,110.7200,44.7590,81.220,4.4940,5.13050,b'1'
5907,-0.179050,1.25530,-0.275990,0.74554,-120.4400,-0.179050,-0.154930,-0.26018,1.17490,-0.326590,...,0.148880,0.548240,0.85112,-0.522430,9.8526,3.4892,207.870,1.7559,9.95270,b'1'
5908,-0.108860,0.74394,0.015449,1.08780,-17.0030,-0.108860,-0.109180,0.12531,0.84516,0.093224,...,-0.183200,-1.167700,1.18320,6.092400,13.8860,6.0769,83.122,4.3911,0.95575,b'1'


Data discretization

In [74]:
#df['bucket']=pd.cut(df['marks'],5,labels=['Poor','Below_average','Average','Above_Average','Excellent'])
#['Poor','Below_average','Average','Above_Average','Excellent']
#category_names = ['Subpar', 'Mediocre', 'Fair', 'Good', 'Superior', 'Outstanding', 'Exceptional', 'Remarkable', 'Extraordinary', 'Unparalleled']
category_names = ['C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8', 'C9', 'C10', 'C11', 'C12', 'C13', 'C14', 'C15', 'C16', 'C17', 'C18', 'C19', 'C20']

atttributes = df.columns.values.tolist()

for i in range(len(atttributes)-1):
    #print(atttributes[i])
    #print(df[atttributes[i]].to_numpy())
    df[atttributes[i]]=pd.cut(df[atttributes[i]].to_numpy(),20,labels=category_names)
    
df

,Attr1,Attr2,Attr3,Attr4,Attr5,Attr6,Attr7,Attr8,Attr9,Attr10,...,Attr56,Attr57,Attr58,Attr59,Attr60,Attr61,Attr62,Attr63,Attr64,class
0,C17,C18,C15,C1,C10,C10,C20,C1,C2,C4,...,C20,C19,C1,C8,C1,C1,C1,C1,C1,b'0'
1,C17,C18,C15,C1,C10,C10,C20,C1,C2,C4,...,C20,C19,C1,C8,C1,C1,C1,C1,C1,b'0'
2,C17,C18,C15,C1,C10,C10,C20,C1,C2,C4,...,C20,C19,C1,C8,C1,C1,C1,C1,C1,b'0'
3,C17,C18,C15,C1,C10,C10,C20,C1,C2,C4,...,C20,C19,C1,C8,C1,C1,C1,C1,C1,b'0'
4,C17,C18,C15,C1,C10,C10,C20,C1,C2,C4,...,C20,C19,C1,C8,C1,C1,C1,C1,C1,b'0'
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5905,C17,C18,C15,C1,C10,C10,C20,C1,C2,C4,...,C20,C19,C1,C8,C1,C1,C1,C1,C1,b'1'
5906,C17,C18,C15,C1,C10,C10,C20,C1,C2,C4,...,C20,C19,C1,C8,C1,C1,C1,C1,C1,b'1'
5907,C17,C18,C15,C1,C10,C10,C20,C1,C2,C4,...,C20,C19,C1,C8,C1,C1,C1,C1,C1,b'1'
5908,C17,C18,C15,C1,C10,C10,C20,C1,C2,C4,...,C20,C19,C1,C8,C1,C1,C1,C1,C1,b'1'
